# Plot ROI Contours and Corresponding Whole-Session Activity Traces

What does this script do
------------------------------------

Generates an interactive plot where the user can select which ROI's whole-session activity is visualized. Event occurrences (ie. timing of events during the session) are denoted by dotted vertical lines and are colored based on condition type. The activity traces and event lines can be toggled on/off and the user can zoom in and out of the plot.

How to run this code
------------------------------------

In this jupyter notebook, First find the code block with the comment header called USER-DEFINED VARIABLES. Edit the variables according to your data and output preferences. Then just run all cells in order (shift + enter; or in the dropdown menu: Kernel->Resart & Run All). Please make sure you set the correct sampling rate (fs) in the user-defined variables block of code.

Parameters
------------------------------------

fname_signal : string
    
    Name of file that contains roi activity traces. Must include full file name with extension. Accepted file types: .npy, .csv. IMPORTANT: data dimensions should be rois (y) by samples/time (x)

fname_events : string

    Name of file that contains event occurrences. Must include full file name with extension. Accepted file types: .pkl, .csv. Pickle (pkl) files need to contain a dictionary where keys are the condition names and the values are lists containing samples/frames for each corresponding event. Csv's should have two columns (event condition, sample). The first row are the column names. Subsequent rows contain each trial's event condition and sample in tidy format. See example in sample_data folder for formatting, or this link: https://github.com/zhounapeuw/NAPE_imaging_postprocess/raw/main/docs/_images/napeca_post_event_csv_format.png

fdir : string 

    Root file directory containing the raw tif, tiff, h5 files. IMPORTANT Note: leave off the last backslash, and include the letter r in front of string (treats the contents as a raw string). For example: r'C:\Users\my_user\analyze_sessions'

fname : string

    Session name; by default this is the name of the parent folder that the data resides in, but can be changed by user to be any string. This fname variable is mainly used to name the saved output files.

fs : float

    Sampling rate of data. It is imperative that this value is correct; otherwise the incorrect time windows will be pulled out for each event. If you suspect that 
    the sampling rate (fs) was not set correctly in the NAPECA preprocessing pipeline, go into the saved json file and edit the fs value.

opto_blank_frame : boolean

    if PMTs were blanked during stim, use detected stim times (from preprocessing) to set those frames to NaN
    
num_rois : int or string

    Set to an (n) integer to plot n first ROIs. Can set to 'all' if want to show all ROIs

selected_conditions : list of strings or None

    Specific conditions that the user wants to analyze; needs to be exactly the name of conditions in the events CSV or pickle file

flag_normalization : string or None

    options: 'dff' (delta F over F), 'zscore', 'dff_perc' (delta F/F with baseline F calculated as 25 percentile of whole session's activity. If set to None, no normalization will occur

In [ ]:
import os
import numpy as np
import glob
import pickle
import json
import seaborn as sns
import matplotlib.ticker as ticker
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import plotly.graph_objects as go
#important for text to be detected when importing saved figures into illustrator
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
plt.rcParams["font.family"] = "Arial"

import utils

In [ ]:
"""
USER-DEFINED VARIABLES
"""

def define_params(method = 'single'):
    
    fparams = {}
    
    if method == 'single':
        
        fparams['fname_signal'] = 'VJ_OFCVTA_7_260_D6_neuropil_corrected_signals_15_50_beta_0.8.csv'   # 
        fparams['fname_events'] = 'event_times_VJ_OFCVTA_7_260_D6_trained.csv' # can set to None if you want to plot the signals only
        # fdir signifies to the root path of the data. Currently, the abspath phrase points to sample data from the repo.
        # To specify a path that is on your local computer, use this string format: r'your_root_path', where you should copy/paste
        # your path between the single quotes (important to keep the r to render as a complete raw string). See example below:
        # r'C:\Users\stuberadmin\Documents\GitHub\NAPE_imaging_postprocess\napeca_post\sample_data' 
        fparams['fdir'] = os.path.abspath('./sample_data/VJ_OFCVTA_7_260_D6') 
        fparams['fname'] = os.path.split(fparams['fdir'])[1]

        # set the sampling rate
        fparams['fs'] = 5

        # session info
        fparams['opto_blank_frame'] = False # if PMTs were blanked during stim, set stim times to nan (instead of 0)
        
        # analysis and plotting arguments
        fparams['num_rois'] = 10 # set to 'all' if want to show all cells
        fparams['selected_conditions'] = None # set to None if want to include all conditions from behav data
        fparams['flag_normalization'] = 'dff_perc' # options: 'dff', 'zscore', 'dff_perc', None
        fparams['cond_colors'] = ['steelblue', 'crimson', 'orchid', 'gold']
       
    elif method == 'f2a': # if string is empty, load predefined list of files in files_to_analyze_event

        fparams = files_to_analyze_event.define_fparams()

    elif method == 'root_dir':
        
        pass

    return fparams

fparams = define_params(method = 'single') # options are 'single', 'f2a', 'root_dir'

if fparams['num_rois'] == 'all':
    fparams['num_rois'] = signals.shape[0]

In [ ]:
# functions to normalize traces
def calc_dff_percentile(activity_vec, perc=25):
    perc_activity = np.percentile(activity_vec, perc)
    return (activity_vec-perc_activity)/perc_activity

def calc_zscore(activity_vec, baseline_samples):
    mean_baseline = np.nanmean(data[..., baseline_samples])
    std_baseline = np.nanstd(data[..., baseline_samples])
    return (data-mean_baseline)/std_baseline

In [ ]:
def define_paths(dict_vars, fparams):
    # define load and save paths
    dict_vars['signals_fpath'] = os.path.join(fparams['fdir'], fparams['fname_signal'])
    if fparams['fname_events']:
        dict_vars['events_file_path'] = os.path.join(fparams['fdir'], fparams['fname_events'])

    dict_vars['save_dir']= os.path.join(fparams['fdir'], 'event_rel_analysis')
    utils.check_exist_dir(dict_vars['save_dir']); # make the save directory

    return dict_vars

In [ ]:
def load_normalize_signals(dict_vars, fparams):
    # load time-series data
    signals = utils.load_signals(dict_vars['signals_fpath'])

    if fparams['opto_blank_frame']:
        try:
            glob_stim_files = glob.glob(os.path.join(fparams['fdir'], "{}*_stimmed_frames.pkl".format(fparams['fname'])))
            stim_frames = pickle.load( open( glob_stim_files[0], "rb" ) )
            signals[:,stim_frames['samples']] = None # blank out stimmed frames
            flag_stim = True
            print('Detected stim data; replaced stim samples with NaNs')
        except:
            flag_stim = False
            print('Note: No stim preprocessed meta data detected.')

    if fparams['flag_normalization'] == 'dff':
        dict_vars['signal_to_plot'] = np.apply_along_axis(utils.calc_dff, 1, signals)
    elif fparams['flag_normalization'] == 'dff_perc':
        dict_vars['signal_to_plot'] = np.apply_along_axis(calc_dff_percentile, 1, signals)
    elif fparams['flag_normalization'] == 'zscore':
        dict_vars['signal_to_plot'] = np.apply_along_axis(calc_zscore, 1, signals, np.arange(0, signals.shape[1]))
    else:
        dict_vars['signal_to_plot'] = signals

    return dict_vars

def calc_min_max(signals):
    min_max = [list(min_max_tup) for min_max_tup in zip(np.min(signals,axis=1), np.max(signals,axis=1))]
    min_max_all = [np.min(signals), np.max(signals)]
    
    return min_max_all


def make_timing_info(dict_vars, fparams, signals):

    dict_vars['total_session_time'] = signals.shape[1]/fparams['fs']
    dict_vars['tvec'] = np.round(np.linspace(0, dict_vars['total_session_time'], signals.shape[1]), 2)
    
    return dict_vars



def load_behav(dict_vars, fparams):
    ### load behavioral data and trial info
    # requires define_paths method to be run first 

    glob_event_files = glob.glob(dict_vars['events_file_path']) # look for a file in specified directory
    if not glob_event_files:
        print('{} not detected. Please check if path is correct.'.format(dict_vars['events_file_path']))
    if 'csv' in glob_event_files[0]:
        dict_vars['event_times'] = utils.df_to_dict(glob_event_files[0])
    elif 'pkl' in glob_event_files[0]:
        dict_vars['event_times'] = pickle.load( open( glob_event_files[0], "rb" ), fix_imports=True, encoding='latin1' ) # latin1 b/c original pickle made in python 2

    # identify conditions to analyze
    all_conditions = dict_vars['event_times'].keys()
    conditions = [ condition for condition in all_conditions if len(dict_vars['event_times'][condition]) > 0 ] # keep conditions that have events

    conditions.sort()
    if fparams['selected_conditions']:
        conditions = fparams['selected_conditions']
    dict_vars['conditions'] = conditions
    #dict_vars['cmap_lines'] = get_cmap(len(conditions)) # colors for plotting lines for each condition

    return dict_vars

dict_analysis_vars = {}
dict_analysis_vars = define_paths(dict_analysis_vars, fparams)
dict_analysis_vars = load_normalize_signals(dict_analysis_vars, fparams)
dict_analysis_vars['min_max_all'] = calc_min_max(dict_analysis_vars['signal_to_plot'])
dict_analysis_vars = make_timing_info(dict_analysis_vars, fparams, dict_analysis_vars['signal_to_plot'])
dict_analysis_vars = load_behav(dict_analysis_vars, fparams)

In [ ]:
# Create figure
def make_whole_session_plot(dict_vars, fparams):

    fig = go.Figure()

    # Add traces, one for each slider step
    for idx_roi in np.arange(fparams['num_rois']):
        fig.add_trace(
            go.Scatter(
                visible=False,
                name='Activity ({})'.format(fparams['flag_normalization']),
                line=dict(color="green", width=1.5),
                x=dict_vars['tvec'],
                y=dict_vars['signal_to_plot'][idx_roi,:]))

    if fparams['fname_events']:
        # add vertical lines for events 
        for idx_cond, cond in enumerate(dict_vars['conditions']):
            for idx_ev, event in enumerate(dict_vars['event_times'][cond]):
                # 2nd case used to hide trace duplicates in legend
                if idx_ev==0:
                    fig.add_trace(go.Scatter(x=[event,event],y=[dict_vars['min_max_all'][0],dict_vars['min_max_all'][1]], visible=True,  
                                             mode='lines', 
                                             line=dict(color=fparams['cond_colors'][idx_cond], width=1.5, dash='dash'), showlegend=True, 
                                             legendgroup=cond,
                                             name='{}'.format(cond)))
                else:
                    fig.add_trace(go.Scatter(x=[event,event], y=[dict_vars['min_max_all'][0],dict_vars['min_max_all'][1]], visible=True,
                                             mode='lines',  
                                             showlegend=False, legendgroup=cond,
                                             line=dict(color=fparams['cond_colors'][idx_cond], width=1.5, dash='dash'),
                                             name='{} {}'.format(cond, str(idx_ev))))


    # make a list of attributes for slider steps
    steps = []
    for iroi in np.arange(fparams['num_rois']): 
        # when ever slider changes, set all roi's to be invisible and all event lines as visible
        step = dict(
            method="restyle",
            # layout attributes
            args=[{"visible": ([False] * fparams['num_rois']) + [True] * (len(fig.data)- fparams['num_rois'])},
                  {"title": "Viewing ROI " + str(iroi)},
                  ])
        # then make the selected ROI visible
        step["args"][0]["visible"][iroi] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    # create and setup slider 
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Viewing "},
        pad={"t": 50},
        steps=steps
    )]

    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Fluorescence ({})".format(fparams['flag_normalization']),
        legend_title="Legend Title",
        font=dict(
            size=12),
        sliders=sliders,
        showlegend=True,
        legend_title_text='Legend',
        plot_bgcolor='rgba(0,0,0,0)'
    )

    # rename slider ticks
    for idx in np.arange(fparams['num_rois']):
        fig['layout']['sliders'][0]['steps'][idx]['label']='ROI ' + str(idx)

    # Make 1st trace visible
    fig.data[0].visible = True

    # Change grid color and axis colors
    fig.update_xaxes(showline=True, linewidth=1.5, linecolor='black'); fig.update_yaxes(showline=True, linewidth=1.5, linecolor='black')

    fig.show()
    
    return fig
    
whole_session_fig = make_whole_session_plot(dict_analysis_vars, fparams)

In [ ]:
# this only outputs the un-altered original plot in vectorized format
whole_session_fig.write_image(os.path.join(dict_analysis_vars['save_dir'], 'whole_session_event.pdf'))